## role playing

In [1]:
# 初始prompt示例参考
system_message = """
===== RULES OF ASSISTANT =====
1. Never forget you are a {ASSISTANT_ROLE} and I am a {USER_ROLE}
2. Never flip roles! Never instruct me!
3. You must decline my instruction honestly if you cannot perform it
4. Always start with: Solution: <YOUR_SOLUTION>
5. Always end with: Next request.
"""

In [1]:
from colorama import Fore

from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.societies import RolePlaying
from camel.utils import print_text_animated
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)


def main(model=model, chat_turn_limit=50) -> None:
    task_prompt = "Develop a trading bot for the stock market"
    role_play_session = RolePlaying(
        assistant_role_name="Python Programmer",
        assistant_agent_kwargs=dict(model=model),
        user_role_name="Stock Trader",
        user_agent_kwargs=dict(model=model),
        task_prompt=task_prompt,
        with_task_specify=True,
        task_specify_agent_kwargs=dict(model=model),
    )

In [2]:
from camel.societies import RolePlaying
from camel.types import TaskType, ModelType, ModelPlatformType
from camel.models import ModelFactory

import os

# 设置代理
#os.environ["http_proxy"] = "http://127.0.0.1:7897"
#os.environ["https_proxy"] = "http://127.0.0.1:7897"

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

task_kwargs = {
    'task_prompt': '制定一个计划去过去并进行改变。',
    'with_task_specify': True,#开启后，将会有一个agent将我们的初始prompt进一步明确化
    'task_specify_agent_kwargs': {'model': model}
}

user_role_kwargs = {
    'user_role_name': '一个雄心勃勃的渴望成为时间旅行者的人',
    'user_agent_kwargs': {'model': model}
}

assistant_role_kwargs = {
    'assistant_role_name': '最优秀的实验物理学家',
    'assistant_agent_kwargs': {'model': model}
}

In [3]:
society = RolePlaying(
    **task_kwargs,             # 任务参数
    **user_role_kwargs,        # 指令发送者的参数
    **assistant_role_kwargs,   # 指令接收者的参数
)

camel.agents.chat_agent - INFO - Model Qwen/Qwen2.5-72B-Instruct, index 0, processed these messages:

 [{'role': 'system', 'content': 'You can make a task more specific.'}, 

 {'role': 'user', 'content': 'Here is a task that 最优秀的实验物理学家 will help 一个雄心勃勃的渴望成为时间旅行者的人 to complete: 制定一个计划去过去并进行改变。.
 
 \nPlease make it more specific. Be creative and imaginative.\nPlease reply with the specified task in 50 words or less. Do not add anything else.'}]

In [5]:
def is_terminated(response):
    """
    当会话应该终止时给出对应信息。
    """
    if response.terminated:
        role = response.msg.role_type.name
        reason = response.info['termination_reasons']
        print(f'AI {role} 因为 {reason} 而终止')

    return response.terminated

In [6]:
def run(society, round_limit: int=10):

    # 获取AI助手到AI用户的初始消息
    input_msg = society.init_chat()

    # 开始互动会话
    for _ in range(round_limit):

        # 获取这一轮的两个响应
        assistant_response, user_response = society.step(input_msg)

        # 检查终止条件
        if is_terminated(assistant_response) or is_terminated(user_response):
            break

        # 获取结果
        print(f'[AI 用户] {user_response.msg.content}.\n')
        # 检查任务是否结束
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            break
        print(f'[AI 助手] {assistant_response.msg.content}.\n')

        # 获取下一轮的输入消息
        input_msg = assistant_response.msg

    return None

In [ ]:
run(society)

[AI 用户] Instruction: 研究并确定虫洞技术的基本原理和可行性。
Input: None.

[AI 助手] Solution: 虫洞技术的基本原理和可行性研究涉及多个复杂的物理概念和理论。以下是对这些概念的详细解释：

### 基本原理

1. **广义相对论**：
   - 广义相对论是爱因斯坦在1915年提出的理论，它描述了重力如何通过时空的曲率来影响物体的运动。根据广义相对论，质量和能量可以弯曲时空。
   - 虫洞（Wormhole）是一种假设的时空结构，它连接两个不同的时空点，使得两点之间的距离大大缩短。虫洞的存在和性质可以通过解爱因斯坦场方程来研究。

2. **克鲁斯卡-塞凯赖什坐标系**：
   - 克鲁斯卡-塞凯赖什坐标系是一种用于描述黑洞内外时空的坐标系。通过这种坐标系，可以更清晰地看到虫洞的可能性。克鲁斯卡-塞凯赖什图展示了黑洞内部和外部的时空结构，以及可能存在的虫洞连接。

3. **负能量物质**：
   - 根据广义相对论，保持虫洞开放需要负能量物质或奇异物质。负能量物质具有负的质量和能量密度，这在经典物理学中是不存在的，但在量子力学中可能存在。例如，卡西米尔效应展示了在某些条件下可以产生负能量。

4. **量子纠缠**：
   - 量子纠缠是一种量子现象，其中两个或多个粒子在相互作用后变得彼此关联，即使相隔很远也能瞬间影响对方的状态。有研究表明，量子纠缠可能与虫洞有关，因为它们都涉及到非局域性的信息传递。

### 可行性分析

1. **理论可行性**：
   - 从理论上讲，虫洞的存在是可能的，但目前还没有直接的实验证据。广义相对论允许存在虫洞，但维持虫洞稳定需要负能量物质，而这种物质在自然界中尚未被发现。

2. **技术挑战**：
   - **负能量物质的获取**：目前没有已知的方法可以在宏观尺度上产生足够的负能量物质。量子力学中的卡西米尔效应产生的负能量非常微小，不足以维持虫洞。
   - **虫洞的稳定性和控制**：即使能够产生负能量物质，如何保持虫洞的稳定性和控制其开口位置也是一个巨大的挑战。
   - **时间和空间的精确控制**：前往特定的历史时期（如1905年）需要对时间和空间进行极其精确的控制，这在当前的技术水平下是不可能实现的。

3. **伦理和法律问题**：
   - 时间旅行可能引发一

## 进阶学习

In [ ]:
from camel.societies import RolePlaying
from camel.types import TaskType, ModelType, ModelPlatformType
from camel.models import ModelFactory

import os
from colorama import Fore

# 设置代理
#os.environ["http_proxy"] = "http://127.0.0.1:7897"
#os.environ["https_proxy"] = "http://127.0.0.1:7897"

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

task_kwargs = {
    'task_prompt': '写一本关于AI社会的未来的书。',
    'with_task_specify': True,
    'task_specify_agent_kwargs': {'model': model}
}

user_role_kwargs = {
    'user_role_name': 'AI专家',
    'user_agent_kwargs': {'model': model}
}

assistant_role_kwargs = {
    'assistant_role_name': '对AI感兴趣的作家',
    'assistant_agent_kwargs': {'model': model}
}

society = RolePlaying(
    **task_kwargs,             # 任务参数
    **user_role_kwargs,        # 指令发送者的参数
    **assistant_role_kwargs,   # 指令接收者的参数
    critic_role_name='human',
    with_critic_in_the_loop=True,
    output_language="中文",
)

def is_terminated(response):
    """
    当会话应该终止时给出对应信息。
    """
    if response.terminated:
        role = response.msg.role_type.name
        reason = response.info['termination_reasons']
        print(f'AI {role} 因为 {reason} 而终止')

    return response.terminated

def run(society, round_limit: int=10):

    # 获取AI助手到AI用户的初始消息
    input_msg = society.init_chat()

    # 开始互动会话
    for _ in range(round_limit):

        # 获取这一轮的两个响应
        assistant_response, user_response = society.step(input_msg)

        # 检查终止条件
        if is_terminated(assistant_response) or is_terminated(user_response):
            break

        # 获取结果
        print(Fore.GREEN + f'[AI 用户] {user_response.msg.content}.\n')
        # 检查任务是否结束
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            break
        print(Fore.BLUE + f'[AI 助手] {assistant_response.msg.content}.\n')

        # 获取下一轮的输入消息
        input_msg = assistant_response.msg

    return None

run(society)


> Proposals from AI专家 (RoleType.USER). Please choose an option:
Option 1:
Instruction: 确定书的结构和章节安排，列出主要部分和每个部分的关键点。
Input: None
Option 2:
Input by Kill Switch Engineer.
Option 3:
Stop!!!


## workforce

In [1]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.messages import BaseMessage
from camel.societies.workforce import Workforce
from camel.toolkits import SearchToolkit
from camel.tasks import Task
from camel.toolkits import FunctionTool

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('QWEN_MAX_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_MAX,
    model_config_dict={"temperature": 0},
    api_key=api_key
)

        
# 创建一个 Workforce 实例
workforce = Workforce(description="旅游攻略制作与评估工作组",new_worker_agent_kwargs={'model':model},coordinator_agent_kwargs={'model':model},task_agent_kwargs={'model':model})

In [2]:
search_tool = FunctionTool(SearchToolkit().search_baidu)

search_agent = ChatAgent(
            system_message="""你是一个专业的旅游信息搜索助手。你的职责是:
                1. 搜索目的地的主要景点信息
                2. 搜索当地特色美食信息
                3. 搜索交通和住宿相关信息
                请确保信息的准确性和实用性。""",
            model=model,
            tools=[search_tool],
            output_language='中文'
        )

planner_agent = ChatAgent(
            system_message="""你是一个专业的旅行规划师。你的职责是:
                1. 根据景点分布规划合理的游览顺序
                2. 为每天安排适量的景点和活动
                3. 考虑用餐、休息等时间
                4. 注意不同季节的特点
                请确保行程安排合理且具有可行性。""",
            model=model,
            output_language='中文'
        )

reviewer_agent = ChatAgent(
    system_message="""你是一个经验丰富的旅行爱好者。你的职责是:
        1. 从游客角度评估行程的合理性
        2. 指出可能的问题和改进建议
        3. 补充实用的旅行小贴士
        4. 评估行程的性价比
        请基于实际旅行经验给出中肯的建议。""",
    model=model,
    output_language='中文'
)


In [3]:
# 添加一个执行网页搜索的Agent
workforce.add_single_agent_worker(
    "一个能够执行网页搜索的Agent",    worker=search_agent,
)

Workforce 139930543331584 (旅游攻略制作与评估工作组)

In [4]:
# 添加工作节点
workforce.add_single_agent_worker(
    "负责搜索目的地相关信息",
    worker=search_agent
).add_single_agent_worker(
    "负责制定详细行程规划",
    worker=planner_agent
).add_single_agent_worker(
    "负责从游客角度评估行程",
    worker=reviewer_agent
)

Workforce 139930543331584 (旅游攻略制作与评估工作组)

In [5]:
from camel.tasks import Task

# 创建一个用于测试的任务
task = Task(
    content="规划一个3天的巴黎旅行计划。",
    id="0",  # id可以是任何标记字符串
)

In [6]:
# 使用jupyter来运行代码请打开下面两行注释
import nest_asyncio
nest_asyncio.apply()

# 让 Workforce 处理这个任务
task = workforce.process_task(task)

Worker node 139930543336720 (负责搜索目的地相关信息) get task 0.0: 搜索巴黎的热门景点、餐厅和活动，提供至少5个不同类别的推荐。
Reply from Worker node 139930543336720 (负责搜索目的地相关信息):

由于部分搜索结果不相关或信息不足，我将根据有效内容提供巴黎的热门景点、餐厅和活动推荐。">">**热门景点:**">1. 巴黎市政厅 (Hôtel de Ville de Paris) - 位于巴黎市中心，是巴黎的地标建筑之一。在欧洲遗产日开放参观，可以提前预约。">2. 埃菲尔铁塔 (Eiffel Tower) - 世界著名的巴黎象征，游客可以在塔顶欣赏到整个城市的壮观景色。">3. 卢浮宫 (Louvre Museum) - 世界最大的艺术博物馆之一，收藏有《蒙娜丽莎》等众多世界级艺术品。">4. 圣母院大教堂 (Notre-Dame Cathedral) - 一座哥特式建筑杰作，虽然遭受过火灾但仍然吸引着无数游客前来参观。">5. 凯旋门 (Arc de Triomphe) - 为纪念拿破仑战争胜利而建，位于香榭丽舍大街的一端。">">">**特色美食及餐厅:**">1. 银塔餐厅 (La Tour d'Argent) - 一家历史悠久的豪华餐厅，以其精致的法式菜肴和壮丽的塞纳河景观闻名。">2. 红磨坊 (Moulin Rouge) - 不仅是一个标志性的夜总会，也拥有自己的高品质餐厅，在这里您可以享受精彩的歌舞表演和美味佳肴。">3. L'Avenue - 由Beaumarly集团经营的时尚餐厅，是体验巴黎现代餐饮文化的好去处。">4. Le Matignon - 同样隶属于Beaumarly集团，是一家高档餐厅，适合寻求高端餐饮体验的旅客。">5. 克里斯汀·康斯坦丁餐厅 (Christian Constant's Restaurants) - 该系列包括几家不同风格的小餐馆，提供从传统法国菜到创新料理的各种选择。">">">**活动:**">关于巴黎近期的具体活动信息未能直接获取到最新内容，不过巴黎常年都有很多精彩的文化艺术活动。例如：">- 每年夏季举办的“夏至音乐节”(Fête de la Musique)，全城街头巷尾都能听到免费的现场音乐演出。">- “白夜”(Nu

In [7]:
print(task.result)

根据收集到的信息和建议，这里为您整理了一个优化后的3天巴黎旅行计划：

**第一天：探索历史与文化**
- **上午**：从参观巴黎市政厅开始您的旅程（建议提前预约），之后步行前往圣母院大教堂感受哥特式建筑的魅力。
- **午餐**：推荐在克里斯汀·康斯坦丁餐厅享用传统法国菜，这样可以更早地为下午的行程做准备。
- **下午**：参观卢浮宫，欣赏世界级艺术品如《蒙娜丽莎》等。考虑到博物馆较大，请预留至少3小时的时间，并尽量安排在饭后直接前往以节省时间。
- **晚餐**：尝试L'Avenue提供的现代法餐体验。
- **晚上**：如果精力允许，可以选择参加一场红磨坊的歌舞表演；或者选择在塞纳河畔散步享受巴黎夜景，为第二天保留更多体力。

**第二天：艺术与时尚**
- **上午**：游览埃菲尔铁塔，并登顶观赏城市风光。为了避免排队，建议尽早到达或在线购票。
- **午餐**：银塔餐厅提供精致的法式料理及美丽的塞纳河景致。
- **下午**：漫步至香榭丽舍大街，在那里您可以购物或是参观凯旋门。如果有兴趣了解法国历史，可以考虑进入凯旋门内部参观，但记得留出足够的时间来探索展览。
- **晚餐**：Le Matignon是一个享受高端餐饮的好地方，位于较为繁忙的区域，请提前规划交通方式以免错过预约。
- **夜晚活动**：根据季节的不同，可参与如夏至音乐节、白夜等活动，或者简单地在塞纳河畔散步享受巴黎夜景。

**第三天：休闲与自然**
- **上午**：选择一个您感兴趣的博物馆或画廊参观，比如奥赛博物馆或罗丹美术馆。
- **午餐**：再次回到克里斯汀·康斯坦丁旗下的小餐馆之一，这次尝试不同的风味。
- **下午**：前往凡尔赛宫进行半日游，这可能需要一整个下午的时间来充分探索这座宏伟的宫殿及其花园。建议早上尽早出发，以便有充足的时间游览并返回市区。
- **晚餐**：结束旅程前的最后一顿饭，可以选择一家自己特别喜欢的地方，为这次巴黎之旅画上完美的句号。

请根据实际情况和个人偏好调整每日计划，注意天气预报以及各个地点的具体开放时间。希望这个计划能让您的巴黎之行更加愉快！


### hackathon评审团

In [1]:
import textwrap

from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.tasks import Task
from camel.toolkits import FunctionTool, SearchToolkit
from camel.types import ModelPlatformType, ModelType
from camel.societies.workforce import Workforce

import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('QWEN_MAX_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_MAX,
    model_config_dict={"temperature": 0},
    api_key=api_key
)


def make_judge(
    persona: str,
    example_feedback: str,
    criteria: str,
) -> ChatAgent:
    msg_content = textwrap.dedent(
        f"""\
        你是一个黑客马拉松的评委。
        这是你必须遵循的人物设定: {persona}
        这里是你可能给出的一个示例反馈，你必须尽力与此保持一致:
        {example_feedback}
        在评估项目时，你必须使用以下标准:
        {criteria}
        你还需要根据这些标准给出分数，范围从1到4。给出的分数应类似于3/4、2/4等。
        """  # noqa: E501
    )

    sys_msg = BaseMessage.make_assistant_message(
        role_name="黑客马拉松评委",
        content=msg_content,
    )

    agent = ChatAgent(
        system_message=sys_msg,
        model=model,
    )

    return agent

In [2]:
proj_content = textwrap.dedent(
    """\
    项目名称: 基于CAMEL的自适应学习助手
    你的项目如何解决一个真实的问题: 我们的基于CAMEL的自适应学习助手解决了在日益多样化和快速变化的学习环境中个性化教育的挑战。传统的一刀切教育方法往往无法满足个别学习者的独特需求，导致理解上的差距和参与度降低。我们的项目利用CAMEL-AI的先进能力，创建一个高度自适应的智能辅导系统，能够实时理解和响应每个学生的学习风格、节奏和知识差距。
    解释你的技术以及哪些部分有效: 我们的系统利用CAMEL-AI的上下文学习和多领域应用特性，创建一个多功能的学习助手。核心组件包括:
    1. 学习者档案分析: 使用自然语言处理评估学生的当前知识、学习偏好和目标。
    2. 动态内容生成: 利用CAMEL-AI创建个性化的学习材料、解释和练习题，针对每个学生的需求量身定制。
    3. 自适应反馈循环: 持续分析学生的反应，并实时调整内容的难度和风格。
    4. 多模态集成: 融合文本、图像和互动元素，以满足不同的学习风格。
    5. 进度跟踪: 提供学生学习旅程的详细见解，识别优势和改进领域。
    目前，我们已成功实现学习者档案分析和动态内容生成模块。自适应反馈循环部分功能正常，而多模态集成和进度跟踪功能仍在开发中。
    """  # noqa: E501
)

In [3]:
# 创建助手代理
search_toolkit = SearchToolkit()
search_tools = [
    FunctionTool(search_toolkit.search_baidu),
]

researcher_agent = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="研究员",
        content="你是一名研究人工智能和开源项目的研究员。"
        "你使用网络搜索来保持对最新创新和趋势的了解。",
    ),
    model=model,
    tools=search_tools,
)

# 创建风险投资家评委
vc_persona = (
    '你是一位对项目如何扩展为“独角兽”公司的风险投资家。'
    '你在讲话中夹杂着“颠覆性”、“协同效应”和“市场渗透”等流行词。'
    '你不关心技术细节或创新，除非它直接影响商业模式。'
)

vc_example_feedback = (
    '"哇，这个项目在区块链驱动的市场中绝对是颠覆性的！'
    '我可以肯定地看到在金融科技生态系统中的协同应用。'
    '可扩展性极高——这是革命性的！'
)

vc_criteria = textwrap.dedent(
    """\
    ### **对现实世界使用的适用性 (1-4 分)**
    - **4**: 项目直接解决了一个重要的现实世界问题，并具有明确的可扩展应用。
    - **3**: 解决方案与现实世界挑战相关，但需要更多的完善以便于实际或广泛使用。
    - **2**: 对现实世界问题有一定的适用性，但解决方案并不立即实用或可扩展。
    - **1**: 与现实世界问题几乎没有相关性，需要进行重大更改才能实际使用。
    """  # noqa: E501
)

vc_agent = make_judge(
    vc_persona,
    vc_example_feedback,
    vc_criteria,
)

# 创建资深工程师评委
eng_persona = (
    '你是一位经验丰富的工程师和完美主义者。你非常注重细节，'
    '对任何技术缺陷都持批评态度，无论多么微小。'
    '你评估每个项目时，仿佛它明天就要投入关键系统使用，'
    '因此你的反馈非常全面，但往往也很苛刻。'
)

eng_example_feedback = (
    '这个项目存在严重的代码效率问题。架构不稳定，内存管理不理想。'
    '我期望接近完美的性能，但这个解决方案在压力测试下几乎无法运行。'
    '它有潜力，但距离部署准备还很远。'
)

eng_criteria = textwrap.dedent(
    """\
    ### **技术实施 (1-4 分)**
    - **4**: 技术执行无可挑剔，设计复杂，性能高效，架构稳健。
    - **3**: 技术实施强劲，但可能有改进或进一步发展的空间。
    - **2**: 项目可以运行，但技术限制或效率低下影响了整体性能。
    - **1**: 技术实施差，功能、编码或结构存在重大问题。
    """  # noqa: E501
)

eng_agent = make_judge(
    eng_persona,
    eng_example_feedback,
    eng_criteria,
)

# 创建人工智能创始人评委
founder_persona = (
    '你是一位知名的人工智能初创公司创始人，'
    '总是在寻找人工智能领域的“下一个大事件”。'
    '你重视大胆、富有创意的想法，优先考虑那些突破新领域的项目，'
    '而不是那些改进现有系统的项目。'
)

founder_example_feedback = (
    '这很有趣，但我之前见过类似的方法。'
    '我在寻找一些突破界限、挑战规范的东西。'
    '这个项目最具革命性的部分是什么？让我们看看互联网上的趋势，'
    '以确保这不是已经存在的东西！'
)

founder_criteria = textwrap.dedent(
    """\
    ### **创新 (1-4 分)**
    - **4**: 项目展示了一个突破性的概念或独特的方法，显著偏离现有方法。
    - **3**: 项目展示了对已知解决方案的新颖扭曲或引入了一些创新方面。
    - **2**: 存在一定程度的创新，但项目主要建立在现有想法上，没有重大新贡献。
    - **1**: 几乎没有创新；项目基于标准方法，创造力极少。
    """  # noqa: E501
)

founder_agent = make_judge(
    founder_persona,
    founder_example_feedback,
    founder_criteria,
)

# 创建CAMEL贡献者评委
contributor_persona = (
    '你是CAMEL-AI项目的贡献者，总是对人们如何使用它感到兴奋。'
    '你友善且乐观，总是提供积极的反馈，即使对于仍然粗糙的项目。'
)

contributor_example_feedback = (
    '哦，我喜欢你在这里实现CAMEL-AI的方式！'
    '利用其自适应学习能力真是太棒了，你真的很好地利用了上下文推理！'
    '让我查看一下GitHub README，看看是否还有更多潜在的优化。'
)

contributor_criteria = textwrap.dedent(
    """\
    ### **CAMEL-AI的使用 (1-4 分)**
    - **4**: 出色地集成了CAMEL-AI，充分利用其先进功能，如上下文学习、自适应性或多领域应用。
    - **3**: 良好地使用了CAMEL-AI，但还有机会利用更多的高级功能。
    - **2**: 对CAMEL-AI的使用有限，主要依赖基本功能，而没有充分利用其全部潜力。
    - **1**: CAMEL-AI的集成很少或实施不当，给项目带来的价值很小。
    """  # noqa: E501
)

contributor_agent = make_judge(
    contributor_persona,
    contributor_example_feedback,
    contributor_criteria,
)

In [4]:
workforce = Workforce(
    '黑客马拉松评审团',
    coordinator_agent_kwargs={"model": model},
    task_agent_kwargs={"model": model},
    new_worker_agent_kwargs={"model": model},
)

workforce.add_single_agent_worker(
    '愿景先锋维罗妮卡（评委），一位风险投资家...',
    worker=vc_agent,
).add_single_agent_worker(
    '批判性约翰（评委），一位经验丰富的工程师...',
    worker=eng_agent,
).add_single_agent_worker(
    '创新者艾瑞斯（评委），一位知名的AI初创公司创始人...',
    worker=founder_agent,
).add_single_agent_worker(
    '友好的弗兰基（评委），CAMEL-AI项目的贡献者...',
    worker=contributor_agent,
).add_single_agent_worker(
    '研究员瑞秋（助手），一位进行在线搜索的研究员...',
    worker=researcher_agent,
)

Workforce 140411620623216 (黑客马拉松评审团)

In [5]:
task = Task(
    content="评估黑客马拉松项目。首先，进行一些与项目相关的信息研究，然后每位评委应相应地给出分数。最后，列出每位评委的意见，同时保留评委的独特身份，以及分数和评委姓名，并给出意见的最终总结。",
    additional_info=proj_content,
    id="0",
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

task = workforce.process_task(task)
print(task.result)

Worker node 140410838250736 (研究员瑞秋（助手），一位进行在线搜索的研究员...) get task 0.0: 研究员瑞秋（助手）进行在线搜索，收集关于基于CAMEL的自适应学习助手项目相关的背景资料、技术细节及市场分析。
Reply from Worker node 140410838250736 (研究员瑞秋（助手），一位进行在线搜索的研究员...):

在进行基于CAMEL的自适应学习助手项目的搜索时，我找到了一些相关的资料。但似乎没有直接与项目名称完全匹配的结果，可能是因为该项目是一个假设的或者新兴的项目。尽管如此，我还是找到了一些关于自适应学习助手、CAMEL框架以及AI智能体技术的相关信息，这可以为我们的项目提供背景和技术细节上的参考。\\n1. [Camel实战第二版 第一章 初识Camel - 简书](http://www.baidu.com/link?url=icJQKgrXMuZfVu3p2q0NQ2mwV8meDu5fJ7bX4xiA8MvEyEVyaePzVyF9oqq45fQG)\\n   这个链接可能提供了Apache Camel的基础知识，这是构建企业集成模式的一个开源框架。虽然它不是专门针对教育领域，但其中的设计理念和架构可以为我们创建自适应学习系统提供灵感。\\n2. [全球生成式AI应用全景图:AI应用进入大爆发时代-腾讯新闻](http://www.baidu.com/link?url=Bfqm1G_ziAKRMl0Q9vc9TAE9q9MU7Q8QdNrvfP515kw0bqB2kAB_AuZwkGunIqpXGtG6TfK5TrsdqcwqDaL3fOOnO9vkKFm4Knul99gA1NjJ13TwoM_HVW6BZSzdoYED5htOCKx5T5nDIW1vXCvZSK)\\n   该文章概述了生成式AI在不同领域的应用情况，包括智能助手。这有助于我们理解市场趋势，并且可以帮助定位我们的基于CAMEL的自适应学习助手如何在教育技术市场中找到位置。\\n3. [AI Agent十大研究方向,数字员工、具身智能与之密切相关|算力智库](http://www.baidu.com/link?url=GSC-tqr7TgUou2kGpqQ9Y5AroFCRg_Ijj3W3E0fiUyVETjnrMj